In [1]:
import pexpect
import json
import os

class LeanServer:
    def __init__(self):
        # Get the path where you download repl
        path_to_repl = os.environ.get('PATH_TO_LEAN_REPL')

        # Run the command
        self.proc = pexpect.spawn(
            "lake env lean --run REPL/Main.lean", 
            cwd=path_to_repl, 
            encoding="utf-8")

    def run_code(self, code, env=None, verbose=False):
        if env:
            command = (
                json.dumps(dict(cmd=code, env=env))
            )  # [1:-1] removes single quotes
        else:
            command = (
                '{ "cmd" : "' + repr(code)[1:-1] + '" }'
            )  # [1:-1] removes single quotes

        if verbose: print(command)
        self.proc.sendline(command)
        self.proc.expect_exact(command + "\r\n")
        self.proc.sendline()
        self.proc.expect_exact("\r\n")
        try:
            index = self.proc.expect('env": \d+\}', timeout=20)
            output = self.proc.before + self.proc.match.group()
            if verbose: print(output)
            return json.loads(output)
            
        except pexpect.exceptions.TIMEOUT:
            raise pexpect.exceptions.TIMEOUT

In [5]:
def get_goal(state):
    goal = None
    for msg in state['messages']:
        if msg['data'].startswith('unsolved goals\n'):
            goal = '\n'.join(msg['data'].split('\n')[1:])
            
        elif msg['severity'] == 'error':
            return None
            
    return goal

In [2]:
from pprint import pprint

code = """
import Mathlib.Data.Nat.Prime

theorem test_thm (m n : Nat) (h : m.Coprime n) : m.gcd n = 1 := by {}
"""

lean = LeanServer()
state = lean.run_code(code)
lean.proc.close()
pprint(state)

{'env': 0,
 'messages': [{'data': 'unsolved goals\n'
                       'm n : ℕ\n'
                       'h : Nat.Coprime m n\n'
                       '⊢ Nat.gcd m n = 1',
               'endPos': {'column': 69, 'line': 4},
               'pos': {'column': 68, 'line': 4},
               'severity': 'error'}]}


In [7]:
print(get_goal(state))

m n : ℕ
h : Nat.Coprime m n
⊢ Nat.gcd m n = 1


In [9]:
code = """
import Mathlib.Data.Nat.Prime

theorem test_thm (m n : Nat) (h : m.Coprime n) : m.gcd n = 1 := by 

rw [← h.gcd_eq_one]
"""

lean = LeanServer()
state = lean.run_code(code)
lean.proc.close()

pprint(state)

{'env': 0}


In [10]:
state.get('sorries') == [] and state.get('messages') == []

False

In [11]:
not state.get('sorries') and not state.get('messages')

True